<a href="https://colab.research.google.com/github/pietrzakkuba/natural-language-processing-labs/blob/master/Lab_8_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tworzenie zasobów

Algorytmy wykorzystywane w problemach przetwarzania języka naturalnego opierają najczęściej swoje działanie o analizę dużych korpusów danych. O ile w zadaniach konkursowych często odpowiednie dane są już przygotowane, o tyle tworząc własne eksperymenty, często musimy sami pozyskać dane i przetransformować do użytecznej postaci.

Dzisiejsze laboratoria dotyczyć będą tworzenia korpusów danych.

## Automatyczne pozyskiwanie surowych danych tekstowych
Dotychczas omawiane metody działały na surowym tekście, który transformowany był do odpowiedniej reprezentacji wektorowej (Bag of words, bag of ngrams, embeddingi). Jak zautomatyzować pozyskiwanie takich surowych danych z internetu?

W tej części skupimy się na stworzeniu automatycznego pobieracza danych, który działać będzie w dwóch "obszarach":
<ol>
<li>crawler: moduł odwiedzający kolejne strony internetowy</li>
<li>scraper: moduł ekstrahujący treść z konkretnych stron internetowych</li>
</ol>

Wykorzystajmy do tego dwie biblioteki: 

**urllib** - do odwiedzania stron

**BeautifulSoup** - do parsowania danych (np. w formacie HTML).

## Zadanie1: Napisz prosty ekstraktor danych ze stron WWW odwiedzający kilka podstron
Ekstraktor ma odwiedzić zadaną stronę internetową, pobrać zawartość wszystkich tekstów wewnątrz paragrafów (wewnątrz tagów P zawartych w pobranym dokumencie HTML), a następnie odwiedzić 5 dowolnych linków z tej strony i z nich analogicznie pobrać zawartość.
Łącznie powinniśmy otrzymać dane z 6 adresów internetowch (strona główna + 5 linków ze strony głównej).

Do napisania crawlera przydać się mogą następujące funkcje:

urllib.request.urlopen() - do pobrania zawartości strony
findAll() na obiekcie BeautifulSoup, można ją wykorzystać do przeiterowania po wszystkich tagach danego rodzaju
get_text() - Istnieje duża szansa, że wewnątrz tagów P znajdą się również inne tagi HTML, chcielibyśmy oczyścić 
z nich tekst. Można to zrobić albo z wyrażeniami regularnymi (robiliśmy takie zadanie na pierwszych laboratoriach!), albo użyć właśnie funkcji get_text() z BeautifulSoup

Linki do dokumentacji:
urllib, pobieranie danych: https://docs.python.org/3/howto/urllib2.html
beautifulSoup: https://www.crummy.com/software/BeautifulSoup/bs4/doc/ (przeczytanie QuickStart jest wystarczające do zrobienia tego zadania)


In [59]:
import urllib.request
from bs4 import BeautifulSoup
import re
import random

def get_paragraphs_page(addr):
    with urllib.request.urlopen(addr) as response:
        html_doc = response.read()
    soap = BeautifulSoup(html_doc, 'html.parser')
    return [par.get_text() for par in soap.findAll('p')]

def get_paragraphs(addr):
    with urllib.request.urlopen(addr) as response:
        html_doc = response.read()
    soap = BeautifulSoup(html_doc, 'html.parser')
    links = [first_page] + random.sample([link.get('href') for link in soap.findAll('a', attrs={'href': re.compile('^http')})], 5)
    paragraphs = []
    for link in links:
        paragraphs += get_paragraphs_page(link)
    return paragraphs

print(get_paragraphs('https://en.wikipedia.org/wiki/Grigori_Rasputin'))

['\n', 'Grigori Yefimovich Rasputin (/ræˈspjuːtɪn/;[1] Russian: Григорий Ефимович Распутин [ɡrʲɪˈɡorʲɪj jɪˈfʲiməvʲɪtɕ rɐˈsputʲɪn]; 21 January\xa0[O.S. 9 January]\xa01869\xa0– 30 December\xa0[O.S. 17 December]\xa01916) was a Russian mystic and self-proclaimed holy man who befriended the family of Emperor Nicholas II, the last monarch of Russia, and gained considerable influence in late imperial Russia.\n', 'Rasputin was born to a peasant family in the Siberian village  of Pokrovskoye in the  Tyumensky Uyezd of Tobolsk Governorate (now Yarkovsky District of Tyumen Oblast). He had a religious conversion experience after taking a pilgrimage to a monastery in 1897. He has been described as a monk or as a "strannik" (wanderer or pilgrim), though he held no official position in the Russian Orthodox Church. He traveled to St. Petersburg in 1903 or the winter of 1904–05, where he captivated some church and social leaders. He became a society figure and met the tsar and Tsarina Alexandra in Nove

# Zadanie 2 - CONLL
Dane ustrukturyzowane w formacie CONLL.

Niektóre algorytmy korzystają z dodatkowych metadanych opisujących poszczególne tokeny (słowa). Bardzo popularnym formatem zapisu takich danych jest format CONLL. 

Reprezentacja CONLL polega na tym, że dany tekst dzielony jest na zdania, a następnie każde zdanie dzielone jest na tokeny (tokenizowane). Następnie dla każdego tokenu tworzymy listę opisującą cechy tego tokenu (słowa).
Poniżej przykład wektora opisującego każdy token zadanego tekstu:
<ol>
    <li>ID - numer porządkowy tokenu w zdaniu</li>
    <li>text - tekst tokenu w formie nieprzetworzonej</li>
    <li>Part of Speech tag (POS tag) - informacja o części mowy, która powiązana jest z tym słowem </li>
    <li>is digit - flaga (o wartościach 0 lub 1), która informuje nas czy dany token jest liczbą</li>
    <li>is punct - flaga (o wartościach 0 lub 1), która informuje nas czy dany token jest znakiem interpunkcyjnym</li>
</ol>

Wektory cech dla kolejnych słów zapisywane są pod sobą. **Separatorem cech w wektorze jest pojedyncza spacja.**

**Zdania zwyczajowo oddzielamy od siebie podwójnym znakiem nowej linii.**

Historycznie CONLL był bardzo konkretnym formatem danych w którym mieliśmy z góry narzucone cechy (np. format CONLL-U https://universaldependencies.org/docs/format.html). Liczba cech ewoluowała jednak w czasie i w wielu miejscach CONLL stał się synonimem ogólnego formatu, w którym dobór cech zależy tylko od nas, jednak stałym jest zapis sekwencji tokenów jako sekwencji wierszy w tekście, gdzie każdy wiersz jest listą oddzielonych spacją wartości (cech), a zdania oddzielone są od siebie podwójnym znakiem nowej linii.


### Przykład:

Tekst: Kasia kupiła 2 lizaki: truskawkowy i zielony. Kasia używa Apple IPhone 5 i IPad.

Reprezentacja CONLL **(spacje separujące kolumny zostały zwielokrotnione na potrzeby zwiększenia czytelności)**
<pre>
1 Kasia  RZECZOWNIK 0 0
2 kupiła CZASOWNIK  0 0
3 2      LICZEBNIK  1 0
4 lizaki RZECZOWNIK 0 0
5 .      _          0 1

1 Kasia  RZECZOWNIK 0 0
2 używa  CZASOWNIK  0 0
3 Apple  RZECZOWNIK 0 0
4 IPhone RZECZOWNIK 0 0
5 5      LICZEBNIK  1 0
6 i      SPÓJNIK    0 0
7 iPad   RZECZOWNIK 0 0
8 .      _          0 1
</pre>

**Zadanie**: Napisz funkcję, która z zadanego tekstu w formie surowego tekstu stworzy reprezentację CONLL opisaną wcześniej wymienionymi atrybutami (ID, text, POS-tag, is_digit, is_punct).

Wykorzystaj sentence splitter i tokenizator z NLTK. Do uzyskania informacji o POS-tagach każdego tokenu wykorzystaj funkcję nltk.pos_tag(). W kolumnie związanej z POS-tagiem zapisz pos tag w takiej formie, w jakiej uzyskamy go z funkcji pos_tag (pos_tag() zwraca formy skrótowe, np. 'NN' dla rzeczowników), nie trzeba więc zamieniać napisu "NN" na "RZECZOWNIK".


In [83]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
import nltk
import string
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

def generate_conll(text):
    sentences = sent_tokenize(text)
    for sentence in sentences:
        words = word_tokenize(sentence)
        i = 0
        for tag in pos_tag(words):
            i += 1
            print('{} {} {} {} {}'.format(i, tag[0], tag[1], int(tag[0].isdigit()), int(tag[0] in string.punctuation)))
        print()

generate_conll("Kate uses IPhone 5 and IPad. Kate bought 2 lolipops.")

1 Kate NNP 0 0
2 uses VBZ 0 0
3 IPhone NNP 0 0
4 5 CD 1 0
5 and CC 0 0
6 IPad NNP 0 0
7 . . 0 1

1 Kate NNP 0 0
2 bought VBD 0 0
3 2 CD 1 0
4 lolipops NNS 0 0
5 . . 0 1




Wyobraźmy sobie teraz, że chcielibyśmy wykrywać wzmianki o urządzeniach elektronicznych w tekście. W jaki sposób zakodować informację o (potencjalnie wielotokenowych) nazwach produktów w CONLL, tak, aby później móc wykonać proces uczenia?

Dodajmy w naszym CONLLu dodatkową kolumnę reprezentującą informację o urządzeniach elektronicznych.
Nazwy urządzeń mogą składać się potencjalnie z wielu słów.
Do zakodowania wielotokenowych tekstów używa się najczęściej notacji IOB, gdzie każda literka skrótu oznacza interpretację aktualnego słowa:
<ul>
    <li> B = begin, marker, który mówi, że aktualne słowo to początek nazwy </li>
    <li> I = inside, marker, który mówi, że aktualne słowo to kontynacja nazwy, która rozpoczyna się wystąpieniem wcześniejszego B</li>
    <li> O = outside, marker, który mówi, że aktualne słowo nie jest interesującą nas nazwą (jest poza nią) </li>
</ul>

Po dodaniu nowej kolumny (na końcu) nasz CONLL przybiera postać:

<pre>
1 Kasia  RZECZOWNIK 0 0 O
2 kupiła CZASOWNIK  0 0 O
3 2                 1 0 O
4 lizaki RZECZOWNIK 0 0 O
5 .      _          0 1 O

1 Kasia  RZECZOWNIK 0 0 O
2 używa             0 0 O
3 Apple  RZECZOWNIK 0 0 B
4 IPhone RZECZOWNIK 0 0 I
5 5                 1 0 I
6 i      SPÓJNIK    0 0 O
7 iPad   RZECZOWNIK 0 0 B
8 .      _          0 1 0
</pre>

Zwróćcie Państwo uwagę na ostatnią kolumnę, czytając tekst od góry w dół, wystąpienie literki "B" oznacza początek interesującej frazy (Apple), jeśli zaraz za "B" pojawia się sekwencja oznaczona jako "I" - kolejne tokeny stanowią kontynuację interesującej nas frazy, w tym przypadku 3 tokeny "Apple IPhone 5" tworzą jeden byt. Poza tym widzimy, że "iPad" stanowi osobny, jednotokenowy byt.

Po co rozróżniać pomiędzy "B", "I" i "O", czy nie można uwzględnić tylko dwóch tagów "wewnątrz frazy", "poza frazą"? Teoretycznie można, ale wprowadzimy w ten sposób sytuacje niejednoznaczne. 

Sprawdźmy to na przykładzie sekwencji "XBox Playstation" reprezentującej 2 osobne byty. Używając tagowania IOB nasza sekwencja wyglądałaby tak:

XBox B
PlayStation B

Widzimy więc, że dwa tagi "B" oznaczają dwa początki osobnych fraz. Co jednak gdybyśmy używali tagów "wewnątrz (interesującej nas) frazy", "poza (interesującą nas) frazą"?

XBox "wewnątrz (interesującej nas) frazy"
Playstation "wewnątrz (interesującej nas) frazy"

W tej sytuacji oznaczyliśmy poprawnie oba tokeny jako części interesujących nas fraz. Jednak nie wiemy, czy XBox Playstation to jedna, czy dwie osobne frazy (byty) -- stąd format IOB jest zdecydowanie bezpieczniejszym wyborem.

**Zadanie**: Napisz funkcję, która wygeneruje CONLL z uwzględnieniem tagów IOB dotyczących urządzeń.
Nasza funkcja posiada teraz dodatkowy argument devices, który zawiera listę obiektów, które opisują gdzie (przesunięcie znakowe) znajduje się początek i koniec wzmianek.


In [129]:
import spacy

def generate_CONLL(text, devices=[]):
    starting_points = [token.idx for token in nlp(text)]
    b_list = []
    i_list = []
    for x in range(len(devices)):
        b_list.append(list(devices[x].values())[0])
        i_list.append(list(devices[x].values())[1])
    sentences = sent_tokenize(text)
    j = 0
    for sentence in sentences:
        words = word_tokenize(sentence)
        i = 0
        for tag in pos_tag(words):
            i += 1
            marker = 'O'
            if starting_points[j] in b_list:
                marker = 'B'
            else:
                for k in range(len(devices)):
                    if starting_points[j] > b_list[k] and starting_points[j] < i_list[k]:
                        marker = 'I'
                        break
            print('{} {} {} {} {} {}'.format(i, tag[0], tag[1], int(tag[0].isdigit()), int(tag[0] in string.punctuation), marker))
            j += 1
        print()

# parametr devices to lista słowników w którym mamy informację o numerze znaku na którym fraza się zaczyna i kończy (zobacz: próba wywołania w ostatniej linijce) (litera I z Iphone występuje na 10 znaku)
# Zapoznaj się z dokumentacją SpaCy (obiekt Token), aby zobaczyć jak wydobyć informację o pozycji danego słowa w zdaniu/dokumencie.
    
generate_CONLL("Kate uses IPhone 5 and IPad. Kate bought 2 lolipops.", devices=[{"begin": 10, "end":18}, {"begin": 23, "end": 27}])



1 Kate NNP 0 0 O
2 uses VBZ 0 0 O
3 IPhone NNP 0 0 B
4 5 CD 1 0 I
5 and CC 0 0 O
6 IPad NNP 0 0 B
7 . . 0 1 O

1 Kate NNP 0 0 O
2 bought VBD 0 0 O
3 2 CD 1 0 O
4 lolipops NNS 0 0 O
5 . . 0 1 O



Często chcemy w tekście naraz oznaczać byty, które należą do różnych kategorii, np. lokacje, numery telefonów, daty, wzmianki o osobach. W takich sytuacjach używa się również kodowania IOB jednak wzbogaca się etykiety o odpowiednie informacje używając formatu:

{tag IOB}-{etykieta kategorii}

Stąd daty przyjmują oznaczenia: B-DATE / I-DATE, osoby B-PERSON / I-PERSON, numery telefonów B-PHONENUMBER / I-PHONENUMBER, lokacje: B-LOCATION / I-LOCATION itp. Wiemy zatem czy dany token należy do interesującej nas frazy i do jakiej kategorii przypisana jest ta fraza.